In [1]:
import findspark
findspark.init('/home/ubuntu/spark-2.1.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Iteration4').getOrCreate()

In [2]:
# Reading the datasets
dataset_one = spark.read.csv('./student-math-dataset1.csv', header=True, inferSchema=True)
dataset_two = spark.read.csv('./student-math-dataset2.csv', header=True, inferSchema=True)

In [3]:
# 2.2 Describe the data
print("Dataset one structure: " + str(dataset_one.count()) + " rows, and " + str(len(dataset_one.columns)) + " columns")
print("Dataset two structure: " + str(dataset_two.count()) + " rows, and " + str(len(dataset_two.columns)) + " columns")
print("How many students inside the dataset: " + str(dataset_one.count()))

Dataset one structure: 395 rows, and 31 columns
Dataset two structure: 395 rows, and 4 columns
How many students inside the dataset: 395


In [4]:
print("Data type inside the dataset one")
dataset_one.dtypes

Data type inside the dataset one


[('id', 'int'),
 ('school', 'string'),
 ('sex', 'string'),
 ('age', 'int'),
 ('address', 'string'),
 ('famsize', 'string'),
 ('Pstatus', 'string'),
 ('Medu', 'int'),
 ('Fedu', 'int'),
 ('Mjob', 'string'),
 ('Fjob', 'string'),
 ('reason', 'string'),
 ('guardian', 'string'),
 ('traveltime', 'int'),
 ('studytime', 'int'),
 ('failures', 'int'),
 ('schoolsup', 'string'),
 ('famsup', 'string'),
 ('paid', 'string'),
 ('activities', 'string'),
 ('nursery', 'string'),
 ('higher', 'string'),
 ('internet', 'string'),
 ('romantic', 'string'),
 ('famrel', 'int'),
 ('freetime', 'double'),
 ('goout', 'int'),
 ('Dalc', 'double'),
 ('Walc', 'double'),
 ('health', 'double'),
 ('absences', 'int')]

In [5]:
print("Data type inside the dataset two")
dataset_two.dtypes

Data type inside the dataset two


[('id', 'int'), ('G1', 'int'), ('G2', 'int'), ('G3', 'int')]

In [10]:
# 2.3 Explore the data
print("Dataset one information: ")
dataset_one.describe().show()

Dataset one information: 
+-------+------------------+------+----+------------------+-------+-------+-------+------------------+------------------+-------+-------+----------+--------+------------------+------------------+------------------+---------+------+----+----------+-------+------+--------+--------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|                id|school| sex|               age|address|famsize|Pstatus|              Medu|              Fedu|   Mjob|   Fjob|    reason|guardian|        traveltime|         studytime|          failures|schoolsup|famsup|paid|activities|nursery|higher|internet|romantic|            famrel|          freetime|             goout|              Dalc|              Walc|            health|         absences|
+-------+------------------+------+----+------------------+-------+-------+-------+------------------+------------------+-------+-------+----------+

In [11]:
dataset_two.describe().show()

+-------+------------------+------------------+------------------+------------------+
|summary|                id|                G1|                G2|                G3|
+-------+------------------+------------------+------------------+------------------+
|  count|               395|               395|               395|               395|
|   mean|             198.0| 10.90886075949367|10.713924050632912|10.415189873417722|
| stddev|114.17092449481173|3.3191946715076686|3.7615046595560306|4.5814426109978434|
|    min|                 1|                 3|                 0|                 0|
|    max|               395|                19|                19|                20|
+-------+------------------+------------------+------------------+------------------+



In [12]:
# same as above
# dataset_one.select(dataset_one.columns).describe().show()
# dataset_two.select('G1', 'G2', 'G3').describe().show()

+-------+------------------+------------------+------------------+
|summary|                G1|                G2|                G3|
+-------+------------------+------------------+------------------+
|  count|               395|               395|               395|
|   mean| 10.90886075949367|10.713924050632912|10.415189873417722|
| stddev|3.3191946715076686|3.7615046595560306|4.5814426109978434|
|    min|                 3|                 0|                 0|
|    max|                19|                19|                20|
+-------+------------------+------------------+------------------+



In [14]:
# start ploting


In [ ]:
# use it to check null vlaue
dataset_one.printSchema()
dataset_two.printSchema()